## Spatial Data Analysis using Python and OpenData

Python Pizza Hamburg, December 11th, 2021

Prof. Martin Christen<br/>
mailto:martin.christen@fhnw.ch<br/>
Twitter: @MartinChristen<br/>
LinkedIn: https://www.linkedin.com/in/martinchristen/

### GitHub

This notebook will be available at: https://github.com/martinchristen/pythonpizzahamburg2021

## 1. Installation of required Modules

(This tutorial requires anaconda, if you don't have it yet, download it here: https://www.anaconda.com/download/ ).

If you try without anaconda: good luck!

**This notebook requires Python 3.7**

Some Geo-Modules are still not ported to 3.8+ on all platforms. With 3.7 we're safe on all major operating Systems.<br/> 
(If you have a Mac with M1 processor you still need a little bit patience)

### Installing Modules (conda)

Install the main modules using conda, dependencies will be resolved (gdal etc.)

    
    conda create --name PizzaHamburg python=3.7 jupyterlab -y
    conda activate PizzaHamburg
    conda install matplotlib -y
    conda install shapely -y
    conda install fiona -y
    conda install rasterio -y
    conda install geopandas -y
    conda install descartes
    conda install folium -c conda-forge
    conda install h3-py -c conda-forge

### 1. Introduction to Shapely

https://shapely.readthedocs.io/en/stable/manual.html

![Features](img/Features.png)

Shapely supports the following Features:

* Point
* LineString
* LinearRing          
* Polygon
* MultiLineString
* MultiPoint
* MultiPolygon

Let's construct a Hexagon. (with or without H3)

In [ ]:
import math

radius = 10
coords = []
for i in range(0,6):
    angle = i*60*math.pi/180
    coords.append((radius * math.cos(angle), radius*math.sin(angle)))
coords

Definition for Polygons: first and last point must be same

So let's add it

In [ ]:
# First and last point must be same in polygon
coords.append(coords[0])

In [ ]:
coords

In [ ]:
from shapely.geometry import Polygon

In [ ]:
hexagon = Polygon(coords)

In [ ]:
hexagon

In [ ]:
hexagon.length

In [ ]:
hexagon.area

Let's create another Polygon, this time a triangle

In [ ]:
triangle = Polygon([(-5,0),(0,6),(5,0),(-5,0)])

In [ ]:
triangle  # note: this output is scaled

We can plot the exerior using matplotlib:

In [ ]:
import matplotlib.pyplot as plt

x1,y1 = hexagon.exterior.xy
x2,y2 = triangle.exterior.xy
plt.plot(x1,y1, 'r')
plt.plot(x2,y2, 'b')
plt.axis("equal");

In [ ]:
hexagon.difference(triangle)

We can do:
   * intersection
   * union
   * difference
   * symmetric_difference

Let's move the triangle a bit to the right to test this

In [ ]:
triangle = Polygon([(5,0),(10,6),(15,0),(5,0)])

In [ ]:
hexagon.difference(triangle)

In [ ]:
#intersection
hexagon.intersection(triangle)

In [ ]:
hexagon.union(triangle)

In [ ]:
hexagon.symmetric_difference(triangle)

wkt: Well Known Text - simple way to display 
    
more info: https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry

In [ ]:
hexagon.symmetric_difference(triangle).wkt

### Binary operations on shapes:

- **contains** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **intersects** (Returns True if the boundary and interior of the object intersect in any way with those of the other.)
- **witin** (Returns True if the object’s boundary and interior intersect only with the interior of the other (not its boundary or exterior).
- **touches** (Returns True if the objects have at least one point in common and their interiors do not intersect with any part of the other.)
- **crosses** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **equals** (Returns True if the set-theoretic boundary, interior, and exterior of the object coincide with those of the other.)

In [ ]:
triangle.intersects(hexagon)

### Draw a Polygon (GeoJSON) on Map


Geographic Coordinates

![Coords](img/longitude.gif)

In [ ]:
from shapely.geometry import mapping

geojson = mapping(hexagon)

geojson

In [ ]:
import folium

map_ch = folium.Map(location=[0, 0], zoom_start=7)  
folium.GeoJson(geojson).add_to(map_ch)
map_ch

If you want hexagons all around the world, use h3

<img width="30%" src="http://1fykyq3mdn5r21tpna3wkdyi-wpengine.netdna-ssl.com/wp-content/uploads/2018/06/image12.png" />

In [ ]:
import h3
from shapely.geometry import Polygon, mapping

h3Index = h3.geo_to_h3(0,0, 3);
coords = h3.h3_set_to_multi_polygon([h3Index])
coords = coords[0][0]
coords.append(coords[0])
hexagon = Polygon(coords)
hexagon

## 2. GeoPandas

Let's use the open data set of geonames.org - all cities with population > 5000.

In [ ]:
import pandas as pd

df = pd.read_csv('data/cities5k.csv', encoding="utf-8", sep=",", header=None, low_memory=False)
df.head(3)

### Removing Columns for Demo...

We really have too many columns, to make everything easier, I just reduce to the most important ones and give some column names.
This is all standard pandas...

In [ ]:
df2 = df[[1,4,5,7,14]]
df2.columns = ["name", "lat", "lng", "type", "population"]
df2.head()

In [ ]:
df2.query("name == 'Paris'")

Remove sections, see https://www.geonames.org/export/codes.html for more details

In [ ]:
df2 = df2[df2.type != 'PPLX']
df2.head()

### Creating a GeoPandas Data Frame

We simply need a **geometry** column containing a **shaply geometry**

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

geometry = [Point(pos) for pos in zip(df2['lng'], df2['lat'])]
gdf = gpd.GeoDataFrame(df2, geometry=geometry)

gdf.head()

We can remove the columns "lat" and "lng", it is redundant. 

In [ ]:
gdf = gdf.drop(['lat', 'lng'], axis=1)

In [ ]:
%matplotlib inline
gdf.plot(color='green', markersize=15, figsize=(15,9));

In [ ]:
from shapely.geometry import Point

Basel = Point([7.59348379, 47.570333604])

dist = gdf.distance(Basel)

In [ ]:
gdf_new = gdf.copy()
gdf_new["distance"] = dist

In [ ]:
gdf_new

In [ ]:
s = gdf_new.sort_values(["distance"], ascending=True)
s.head(10)

In [ ]:
big_cities = gdf_new[gdf_new.population > 5000000]
big_cities.head()

### Now display cities on a folium map using markers

In [ ]:
import folium

map_cities = folium.Map(location=[47.570333604, 7.59348379], zoom_start=2)

def create_marker(row):
    lng = row["geometry"].x
    lat = row["geometry"].y
    name = row["name"]
    population = str(int(row["population"]))
    folium.Marker([lat, lng], popup=f'{name}, population:{population}').add_to(map_cities)
    
big_cities.apply(create_marker, axis=1)
map_cities


### 3. Live Data & GeoPandas

We're looking at the earthquake data from USGS:
https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

This data is updated every minute

In [ ]:
import requests

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson"

data = requests.get(url)
file = open("earthquakes.geojson","wb")
file.write(data.content)
file.close()

In [ ]:
import geopandas as gpd

eq_gdf = gpd.read_file("earthquakes.geojson")
eq_gdf.head()

Let's simplify the output and only take most important rows

In [ ]:
eq = eq_gdf[["time","mag", "place","geometry"]].copy()
eq.head()

Look at the histogramm:

In [ ]:
eq.mag.hist(bins=16);

Timestamps in UTC are not really human readable...
Let's convert them

In [ ]:
from datetime import datetime, timezone

data = []
for row in range(0,len(eq)):
    time = eq.iloc[row].time
    t = str(datetime.fromtimestamp(time/1000.0, timezone.utc))
    data.append(t)
    
eq["time_utc"] = data
eq.head()

In [ ]:
eq = eq.drop(['time'], axis=1)

In [ ]:
eq.plot();

Open Natural Earth Dataset with all Polygons of all countries

In [ ]:
gdfAdmin0 = gpd.read_file("data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp", encoding="utf-8")
gdfAdmin0.head()

In [ ]:
countries = gdfAdmin0.plot(figsize=(15,9), color="black")
eq.plot(ax=countries, color="red", markersize=10);

In [ ]:
eq.sort_values(["mag"], ascending=False).head()

### 4. More GeoPandas Fun

In [ ]:
import geopandas as gpd

gdfAdmin0 = gpd.read_file("data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp", encoding="utf-8")
gdfAdmin0.head()

ch = gdfAdmin0[gdfAdmin0['NAME'] == "Switzerland"]
ch

In [ ]:
ch.plot();

In [ ]:
import folium

center = [47.570333604, 7.59348379] 
map_ch = folium.Map(center, zoom_start=5)   

folium.GeoJson(ch).add_to(map_ch)

map_ch

In [ ]:
import folium

center = [47.570333604, 7.59348379] 
map = folium.Map(center, zoom_start=5)   

folium.GeoJson(ch,style_function=lambda feature: {
        'fillColor': 'green',   # you can also replace this with functions with feature as argument
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }).add_to(map)

map

### 5. Raster Data with rasterio



In [ ]:
import rasterio

dataset = rasterio.open('data/BlueMarble.tif', 'r')

In [ ]:
dataset.name

In [ ]:
dataset.count # number of raster bands, in our case 3 for r,g,b

In [ ]:
dataset.width, dataset.height

In [ ]:
dataset.crs

In [ ]:
dataset.bounds

In [ ]:
dataset.transform  # affine transformation pixel to crs

In [ ]:
dataset.transform * (0, 0)    # Pixel to CRS

In [ ]:
~dataset.transform # inverse affine transformation

In [ ]:
~dataset.transform * (0,0) # CRS to Pixel

In [ ]:
px, py = ~dataset.transform * (7.59348379, 47.570333604) # Our Location to Pixel (lng/lat)
print(px,py)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
r = dataset.read(1)
g = dataset.read(2)
b = dataset.read(3)

In [ ]:
rgb = np.dstack((r,g,b))  # stack r,g,b so we can display it...

In [ ]:
fig, ax = plt.subplots(figsize=(15,9))
ax.imshow(rgb, interpolation='nearest')
ax.plot(px,py, 'ro'); 

## GeoPython 2022

Yes, there will be an GeoPython 2022. And yes, it will be in person/hybrid.<br/>
The number of in-person attendees may be limited, we will be ready in January with a decision.<br/>


<img src="img/FlyerGeoPython2022.png" />


<br/>

* Follow @GeoPythonConf on Twitter: http://twitter.com/GeoPythonConf
* The website will be available soon, probably January.
* Talk submission starts in January too.


